In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm
from functools import partial
from sqlalchemy import MetaData, Table, create_engine
from sqlalchemy.sql import text

from config import db_connection_string
from utils import tx_count_between_accounts, tx_count_between_grants
from iou_calculator import IoUCalculator


In [2]:
engine = create_engine(db_connection_string)

In [3]:
engine.execute("""
    CREATE TEMPORARY TABLE `account`
    (INDEX `account_index` (`account`))
    SELECT 
        `account`, 
        COUNT(DISTINCT `grant`) AS `distinct_grant_count`
    FROM `contribution`
    WHERE `grant`!='86'
    GROUP BY `account`;
            
    CREATE TEMPORARY TABLE `grant_contributor`
    (INDEX `account_index` (`account`))
    SELECT DISTINCT 
        `grant`, 
        `account`
    FROM `contribution`
    WHERE `grant`!='86';
""")


In [4]:
grant_contributor_df = pd.read_sql_query("""
    SELECT 
        `grant_contributor`.`grant`, 
        `account`.`account` AS `contributor`, 
        `account`.`distinct_grant_count` AS `contributor_distinct_grant_count`
    FROM `grant_contributor`
    INNER JOIN `account` ON `account`.`account`=`grant_contributor`.`account`
""", engine)


In [5]:
pd.DataFrame([
    [
        contributed_grants_less_than_equal,
        round(ratio / 10, 1),
        (
            grant_contributor_df.groupby('grant').agg(
                ratio=('contributor_distinct_grant_count', lambda x: (
                    x <= contributed_grants_less_than_equal).sum() / x.shape[0])
            )['ratio'] >= ratio / 10
        ).sum()
    ]
    for contributed_grants_less_than_equal in range(1, 11)
    for ratio in range(1, 11)
], columns=['account contributed grants<=', 'matched account ratio>=', 'count']).pivot(index='account contributed grants<=', columns='matched account ratio>=', values='count')


matched account ratio>=,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
account contributed grants<=,,,,,,,,,,
1,233,145,87,51,35,18,11,8,8,7
2,351,219,140,91,70,35,21,17,16,15
3,419,261,169,122,84,48,31,24,19,18
4,492,305,191,135,97,55,36,27,22,20
5,571,349,224,150,114,62,42,30,25,23
6,637,394,256,172,125,71,47,34,28,26
7,685,433,273,180,133,75,50,35,29,27
8,729,486,299,195,142,86,54,38,31,29
9,772,546,345,217,151,92,56,38,31,29
